
# Выбросы и новизна

https://github.com/yzhao062/anomaly-detection-resources

Строго говоря, в анализе данных есть два направления, которые занимаются поиском аномалий:
- поиск выбросов (Outlier Detection)
- поиск «новизны» (Novelty Detection).

Как и выброс, «новый объект» — это объект, который отличается по своим свойствам от объектов (обучающей) выборки.
Но  в отличие от выброса, его в самой выборке пока нет (он появится через некоторое время, и задача как раз и заключается в том,
чтобы обнаружить его при появлении). Например, если вы анализируете замеры температуры и отбрасываете аномально большие или маленькие,
то Вы боретесь с выбросами. А если Вы создаёте алгоритм, который для каждого нового замера оценивает, насколько он похож на прошлые, и выбрасывает
аномальные — Вы «боретесь с новизной».

*Выбросы* являются следствием:

- ошибок в данных (неточности измерения, округления, неверной записи и т.п.)
- наличия шумовых объектов (неверно классифицированных объектов)
- присутствия объектов «других» выборок (например, показаниями сломавшегося датчика).

Новизна, как правило, появляется в результате принципиально нового поведения объекта. Скажем, если наши объекты – описания работы системы, то после проникновения в неё вируса объекты становятся «новизной». Ещё пример – описания работы двигателя после поломки. Здесь важно понимать, что «новизна» называется новизной по той причине, что такие описания для нас абсолютно новые, а новые они потому, что мы не можем в обучающей выборке иметь информацию о всевозможных заражениях вирусами или всевозможных поломках. Формирование такой обучающей выборки трудозатратно и часто не имеет смысла. Зато можно набрать достаточно большую выборку примеров нормальной (штатной) работы системы или механизма.

Приложений здесь море:

- Обнаружение подозрительных банковских операций (Credit-card Fraud)
- Обнаружение вторжений (Intrusion Detection)
- Обнаружение нестандартных игроков на бирже (инсайдеров)
- Обнаружение неполадок в механизмах по показаниям датчиков
- Медицинская диагностика (Medical Diagnosis)
- Сейсмология

# Основные методы обнаружения выбросов / аномалий

1. Статистические тесты
2. Модельные тесты
3. Итерационные методы
4. Метрические методы
5. Метод подмены задачи
6. Методы машинного обучения
7. Ансамбли алгоритмов

## Статистические тесты

Тесты, направленные на поиски экстремальных значений по каждой переменной:
- z-value
- kurtosis measure
- boxplot
- правило 3 сигма

Важно понимать, что экстремальное значение и аномалия это разные понятия. Например, в небольшой выборке

[1, 39, 2, 1, 101, 2, 1, 100, 1, 3, 101, 1, 3, 100, 101, 100, 100]
значение 39 можно считать аномалией, хотя оно не является максимальным или минимальным.
Также стоит отметить, что аномалия характеризуется, как правило, не только экстремальными значениями отдельных признаков

## Модельные тесты

Идея очень простая – мы строим модель, которая описывает данные.
Точки которые сильно отклоняются от модели (на которых модель сильно ошибается) и есть аномалии.
При выборе модели мы можем учесть природу задачи, функционал качества и т.п.

Такие методы хороши для определения новизны, но хуже работают при поиске выбросов.
Действительно, при настройке модели мы используем данные, в которых есть выбросы (и она под них «затачивается»).

## Итерационные методы

Методы, которые состоят из итераций, на каждой из которых удаляется группа «особо подозрительных объектов».
Например, в n-мерном признаковом пространстве можно удалять выпуклую оболочку наших точек-объектов,
считая её представителей выбросами.
Как правило, методы этой группы достаточно трудоёмки.

## Метрические методы

В них постулируется существование некоторой метрики в пространстве объектов, которая и помогает найти аномалии.
Интуитивно понятно, что у выброса мало соседей, а у типичной точки много.
Поэтому хорошей мерой аномальности может служить, например «расстояние до k-го соседа» (см. метод Local Outlier Factor).
Здесь используются специфические метрики, например расстояние Махаланобиса.


## Методы подмены задачи

Когда возникает новая задача, есть большой соблазн решить её старыми методами (ориентированными на уже известные задачи).
Например, можно сделать кластеризацию, тогда маленькие кластеры, скорее всего, состоят из аномалий.
Если у нас есть частичная информация об аномалиях,
то можно решить её как задачу классификации с классами 1 (размеченные аномалии) и 0 (все остальные объекты).
Если бы класс 0 состоял только из нормальных объектов, то такое решение было бы совсем законным.
В противном случае остаётся надеяться, что недетектированных аномалий в нём немного.

## Методы машинного обучения

### Метод опорных векторов для одного класса (OneClassSVM)

По сути это обычный SVM, который отделяет выборку от начала координат, где в качестве ядра берут только rbf - радиально базисные функции.
Именно этот метод помогал долгие годы решать задачу обнаружения поломок сложных механизмов.
Полезно помнить, что OneClassSVM это скорее алгоритм поиска новизны, а не выбросов, т.к. «затачивается» под обучающую выборку.

Важные параметры реализации ***sklearn.svm.OneClassSVM***:

- kernel – ядро (линейное: linear, полиномиальное: poly, радиальные базисные функции: rbf, сигмоидальное: sigmoid, своё заданное)
- nu – верхняя граница на %ошибок и нижняя на % опорных векторов (0.5 по умолчанию)
- degree – степень для полиномиального ядра
- gamma – коэффициент для функции ядра (1/n_features по умолчанию)
- coef0 – параметр в функции полиномиального или сигмоидального ядра

### Изолирующий лес (IsolationForest)

Является одной из вариаций случайного леса:

- Состоит из деревьев
- Каждое дерево строится до исчерпании выборки
- Для построения ветвления в дереве: выбирается случайный признак и случайное расщепление
- Для каждого объекта мера его нормальности – среднее арифметическое глубин листьев, в которые он попал (изолировался)

Логика алгоритма простая: при описанном «случайном» способе построения деревьев выбросы будут попадать в листья на ранних этапах
(на небольшой глубине дерева), т.е. выбросы проще «изолировать»
(напомним, что дерево строится до тех пор, пока каждый объект не окажется в отдельном листе).
Алгоритм хорошо отлавливает именно выбросы

Важные параметры реализации ***sklearn.ensemble.IsolationForest***

- n_estimators – число деревьев
- max_samples – объём выборки для построения одного дерева (если вещественное число, то процент всей выборки)
- contamination – доля выбросов в выборке (для выбора порога)
- max_features – число (или %) признаков, которые используются при построении одного дерева (пока работает только со значением 1.0)
- bootstrap – включение режима бутстрепа при формировании подвыборки

### Эллипсоидальная аппроксимация данных (EllipticEnvelope)

Эллипсоидальная аппроксимация данных — из названия понятно, что облако точек моделируется как внутренность эллипсоида.
Метод хорошо работает только на одномодальных данных, а совсем хорошо – на нормально распределённых.
Степень новизны здесь фактически определяется по расстоянию Махаланобиса.

# PyOD: унифицированная библиотека для обнаружения выбросов

https://pyod.readthedocs.io/en/latest/

In [5]:
from pyod.utils.data import generate_data
import numpy as np
# Смоделируем датасет, где 10% точек - выбросы / аномалии
X_train, X_test, y_train, y_test=generate_data(n_train=200,
                                               n_test=100,
                                               n_features=5,
                                               contamination=0.1,
                                               random_state=3,
                                               behaviour="new")
X_train = X_train * np.random.uniform(0, 1, size=X_train.shape)
X_test = X_test * np.random.uniform(0,1, size=X_test.shape)

In [10]:
# ABOD (Angle-based Outlier Detector)
from pyod.models.abod import ABOD
clf_name = 'ABOD'
clf = ABOD()
clf.fit(X_train)
test_scores = clf.decision_function(X_test)

from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score, recall_score
roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
prn = round(precision_n_scores(y_test, test_scores), ndigits=4)
print(f'{clf_name} ROC:{roc}, precision @ rank n:{prn}')

ABOD ROC:0.7944, precision @ rank n:0.6


In [11]:
# COPOD
from pyod.models.copod import COPOD
clf_name = 'COPOD'
clf = COPOD()
clf.fit(X_train)
test_scores = clf.decision_function(X_test)

roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
prn = round(precision_n_scores(y_test, test_scores), ndigits=4)
print(f'{clf_name} ROC:{roc}, precision @ rank n:{prn}')

COPOD ROC:0.9611, precision @ rank n:0.8


In [12]:
from pyod.models.iforest import IForest
clf_name = "forest"
clf = IForest()
clf.fit(X_train)

test_scores = clf.decision_function(X_test)
roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
prn = round(precision_n_scores(y_test, test_scores), ndigits=4)
print(f'{clf_name} ROC:{roc}, precision @ rank n:{prn}')

forest ROC:0.8967, precision @ rank n:0.7


# PyCaret

Содержит также много готовых реализаций методов поиска аномалий

Классный ноутбук со всеми методами еще раз
https://nbviewer.org/github/DmitrySerg/otus-public/blob/master/OpenLessons/AnomalyDetection/AnomalyDetection_OpenLesson.ipynb